In [57]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Eindopdracht Inleiding Vision
# Tom Schoonbeek & Djim Oomes

# Webcam
cam = cv2.VideoCapture(0)

# Capture Window
cv2.namedWindow("Bottle Capture")

img_counter = 0

# Running loop
while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to handshake with camera")
        break
    cv2.imshow("Bottle Capture", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC button press
        print("ESC pressed, closing Bottle Capture application")
        break
    elif k%256 == 32:
        # SPACE button press
        img_name = "bottle_capture_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written to file".format(img_name))
        
# Vision logic
        # Foto inladen die net is opgeslagen
        #bottle_3_channel = cv2.imread(img_name)
        
        # Voor sample image ipv webcam onderstaande regel mee laten doen en bovenste regel uitcommenten:
        bottle_3_channel = cv2.imread('bottle_capture_sample.png')
        
        bottle_gray = cv2.split(bottle_3_channel)[0]
        
        # Gaussian Blur
        bottle_gray = cv2.GaussianBlur(bottle_gray, (3, 3), 0)
        
        # Manual threshold + contour fles
        threshold_value = 198.5
        (T, bottle_threshold) = cv2.threshold(bottle_gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
        contour_bottle = cv2.morphologyEx(bottle_threshold,cv2.MORPH_GRADIENT,kernel)
        cv2.imshow("Contour fles", contour_bottle)
        cv2.waitKey(0)
        
        # Invert image voor Erosion & Opening
        bottle_threshold = cv2.bitwise_not(bottle_threshold)
        
        # Erosion
        kernel = np.ones((3,3),np.uint8)
        bottle_eroded = cv2.erode(bottle_threshold,kernel,iterations = 3)
        
        # Opening - Hierna is inhoud fles en flesdop normaalgesproken geïsoleerd
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        opening_internal = cv2.morphologyEx(bottle_eroded, cv2.MORPH_OPEN, kernel)
        
        # Invert image terug naar origineel
        opening_internal = cv2.bitwise_not(opening_internal)
        
        # Contours
        image, contours, hierarchy = cv2.findContours(opening_internal, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
        
        # Internal contours (vloeistof+flesdop)
        internal_contours = np.zeros(image.shape)
        for i in range(len(contours)):
            if hierarchy[0][i][3] != -1: # checkt laatste value uit elke row van de contours array op value die niet -1 is
                cv2.drawContours(internal_contours, contours, i, 255, -1)
        # Sorteren per grootte van area, pakt de eennagrootste waarde voor de flesinhoud (grootste waarde is de achtergrond) en de tweenagrootste voor flesdop
        areas = [cv2.contourArea(contour) for contour in contours]
        (contours, areas) = zip(*sorted(zip(contours, areas), key=lambda a:a[1]))
        
        # Inhoud fles
        i = len(contours)-2
        bottle_clone = bottle_3_channel.copy()
        cv2.drawContours(bottle_clone, [contours[i]], -1, (255, 0, 0), 2)
        cv2.imshow("Contour flesinhoud", bottle_clone)
        cv2.waitKey(0)
        
        # Flesdop
        j = len(contours)-3
        bottle_clone = bottle_3_channel.copy()
        cv2.drawContours(bottle_clone, [contours[j]], -1, (255, 0, 0), 2)
        cv2.imshow("Contour flesdop", bottle_clone)
        cv2.waitKey(0)
        
        # Bounding box maken voor berekenen aspect ratio van inhoud fles en goed- en afkeuren
        bottle_clone = bottle_3_channel.copy()
        (x, y, w, h) = cv2.boundingRect(contours[i]) #i vanwege inhoud fles
        aspectRatio = w / float(h)
        
        # Vulpercentage 
        if aspectRatio < 0.380:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te hoog", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "100%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        elif aspectRatio > 0.380 and aspectRatio < 0.421940928:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te hoog", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "90-100%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        elif aspectRatio > 0.421940928 and aspectRatio < 0.474683544:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(bottle_clone, "Correct", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
            cv2.putText(bottle_clone, "80-90%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)
        elif aspectRatio > 0.474683544 and aspectRatio < 0.542495479:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "70-80%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        elif aspectRatio > 0.542495479 and aspectRatio < 0.632911392:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "60-70%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        elif aspectRatio > 0.632911392 and aspectRatio < 0.759493671:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "50-60%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        elif aspectRatio > 0.759493671:
            cv2.rectangle(bottle_clone, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(bottle_clone, "Te laag", (x + 10, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
            cv2.putText(bottle_clone, "<50%", (x + 10, y + 40), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)
        cv2.imshow("Controle flesinhoud", bottle_clone)
        cv2.waitKey(0)
        
cam.release()

cv2.destroyAllWindows()

bottle_capture_0.png written to file
ESC pressed, closing Bottle Capture application
